<a href="https://colab.research.google.com/github/denisangelo/Federated_Learning/blob/main/DP2_FL_Not_Division_Dataset_Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importanto Pytorch, Flower,Opacus
!pip install -q flwr[simulation]
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install opacus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#Importanto as bibliotecas necessárias
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional

import flwr as fl
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import opacus
from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager
from flwr.common import Metrics
from torch.optim.adam import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
import warnings
warnings.simplefilter("ignore")

In [3]:
#Definindo o dispositivo de treinamento
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE} using Pytorch{torch.__version__}, Flower{fl.__version__} and Opacus{opacus.__version__ }")

Training on cpu using Pytorch2.0.1+cu118, Flower1.4.0 and Opacus1.4.0


In [4]:
# Definindo os valores das variáveis
BATCH_SIZE = 1024
MAX_PHYSICAL_BATCH_SIZE = 512
NUM_CLIENTS =10
num_rounds =10
optim_lr=0.001
local_epochs =5
epsilon=50.0
delta=1e-5
max_grad_norm=1.2
step_size = 1
step_gamma=0.6

In [5]:
#Carregando os dados
def load_data():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = CIFAR10(root="./data", train=True,download=True, transform=transform)
    testset = CIFAR10(root="./data", train=False, download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,shuffle=True,num_workers=0)
    testloader = DataLoader(testset,batch_size=BATCH_SIZE,shuffle=False,num_workers=0)
    num_examples = {"trainset": len(trainset)*BATCH_SIZE, "testset": len(testset)*BATCH_SIZE}

    return trainloader, testloader, num_examples

In [6]:
# Definindo as funções de treinamento
def train(net, trainloader, epochs):
    net.train()
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(),lr=optim_lr,)
    sheduler =StepLR(optimizer,step_size=step_size,gamma=step_gamma)
    privacy_engine = PrivacyEngine()
    net, optimizer, trainloader = privacy_engine.make_private_with_epsilon(
    module=net,
    optimizer=optimizer,
    data_loader=trainloader,
    epochs=local_epochs,
    target_epsilon=epsilon,
    target_delta=delta,
    max_grad_norm=max_grad_norm,

)
    for _ in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        with BatchMemoryManager(
            data_loader=trainloader, 
            max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE, 
            optimizer=optimizer) as memory_safe_data_loader:
            for i, (images, labels) in enumerate(memory_safe_data_loader):
                optimizer.zero_grad()
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                loss = criterion(net(images), labels)
                loss.backward()
                optimizer.step()
                if (i+1) % 100 == 99:
                   eps= privacy_engine.get_epsilon(delta)
            sheduler.step()
          
def test(net, testloader):
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [7]:
# Definindo o Modelo
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
# Funções de atualização de parâmetros do modelo
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

In [9]:
# Definindo a classe de cliente Virtual
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net,trainloader,testloader,num_examples):
        super().__init__()
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.testloader = testloader
        self.num_examples = num_examples

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net,self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.testloader)
        return float(loss), len(self.testloader), {"accuracy": float(accuracy)}

def client_fn(cid) -> FlowerClient:
    # Load model
    net = Net().to(DEVICE)
    # Load data (CIFAR-10)
    trainloader,testloader, num_examples= load_data()

    return FlowerClient(cid, net,trainloader, testloader,num_examples) 


In [10]:
# Realizando a agregação
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

In [11]:
# Iniciando o treino federado
# Criando a Função de agregação FedAvg
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,#-->Fração de clientes necessários para treinamento
    fraction_evaluate=1.0, #--> Fração de clientes que serão utilizados para teste
    min_fit_clients=NUM_CLIENTS, #--> Número mínimo que serão utlilizados para treinamento
    min_evaluate_clients=2, #--> Número mínimo de clientes que serão realizados para teste
    min_available_clients=NUM_CLIENTS,#--> Número mínimo de clientes que serão utilizados no treinamento
    evaluate_metrics_aggregation_fn=weighted_average,  
)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}
# Inicia a simulação
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-06-11 08:53:18,992 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-06-11 08:53:23,548	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-11 08:53:27,536 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3888482304.0, 'memory': 7776964608.0, 'node:172.28.0.12': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 3888482304.0, 'memory': 7776964608.0, 'node:172.28.0.12': 1.0}
INFO flwr 2023-06-11 08:53:27,546 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-11 08:53:27,554 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(pid=2700) 2023-06-11 08:53:33.255982: W tensorflow/compiler

(launch_and_get_parameters pid=2700) Files already downloaded and verified
(launch_and_get_parameters pid=2700) Files already downloaded and verified


INFO flwr 2023-06-11 08:53:41,207 | server.py:277 | Received initial parameters from one random client
INFO:flwr:Received initial parameters from one random client
INFO flwr 2023-06-11 08:53:41,223 | server.py:88 | Evaluating initial parameters
INFO:flwr:Evaluating initial parameters
INFO flwr 2023-06-11 08:53:41,225 | server.py:101 | FL starting
INFO:flwr:FL starting
DEBUG flwr 2023-06-11 08:53:41,228 | server.py:218 | fit_round 1: strategy sampled 10 clients (out of 10)
DEBUG:flwr:fit_round 1: strategy sampled 10 clients (out of 10)


(launch_and_get_parameters pid=2700) [Client 4] get_parameters
(launch_and_fit pid=2700) Files already downloaded and verified
(launch_and_fit pid=2700) Files already downloaded and verified
(launch_and_fit pid=2700) [Client 8] fit, config: {}


(launch_and_fit pid=2700) /usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
(launch_and_fit pid=2700)   warnings.warn(
(launch_and_fit pid=2700) /usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
(launch_and_fit pid=2700)   warnings.warn(
(launch_and_fit pid=2700) /usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
(launch_and_fit pid=2700)   z = np.log((np.exp(t) + q - 1) / q)
(pid=2701) 2023-06-11 08:53:44.942960: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find Ten

...

(launch_and_fit pid=2700) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=2701) Files already downloaded and verified
(launch_and_fit pid=2700) [Client 7] fit, config: {}
(launch_and_fit pid=2700) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_fit pid=2701) Files already downloaded and verified
(launch_and_fit pid=2700) [Client 6] fit, config: {}
(launch_and_fit pid=2701) Files already downloaded and verified [repeated 3x across cluster]
(launch_and_fit pid=2701) Files already downloaded and verified
(launch_and_fit pid=2700) Files already downloaded and verified
(launch_and_fit pid=2700) Files already downloaded and verified
(launch_and_fit pid=2700) [Client 1] fit, config: {}
(launch_and_fit pid=2700) Files already downloaded and verified [repeated 3x across cluster]
(launch_and_fit pid=2701) Files already downloaded and verified
(launch_and_fit pid=2700) [Client 8] fit, config: {}
(launch_and_fit pid=2700) F

DEBUG flwr 2023-06-11 17:14:31,791 | server.py:232 | fit_round 10 received 10 results and 0 failures
DEBUG:flwr:fit_round 10 received 10 results and 0 failures
DEBUG flwr 2023-06-11 17:14:31,832 | server.py:168 | evaluate_round 10: strategy sampled 10 clients (out of 10)
DEBUG:flwr:evaluate_round 10: strategy sampled 10 clients (out of 10)


(launch_and_evaluate pid=2701) Files already downloaded and verified [repeated 2x across cluster]
(launch_and_evaluate pid=2701) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=2701) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=2700) Files already downloaded and verified
(launch_and_evaluate pid=2701) [Client 4] evaluate, config: {}
(launch_and_evaluate pid=2701) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=2700) Files already downloaded and verified
(launch_and_evaluate pid=2701) [Client 8] evaluate, config: {}
(launch_and_evaluate pid=2701) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=2700) Files already downloaded and verified
(launch_and_evaluate pid=2701) [Client 9] evaluate, config: {}
(launch_and_evaluate pid=2701) Files already downloaded and verified [repeated 4x across cluster]
(launch_and_evaluate pid=2700) Files already downlo

DEBUG flwr 2023-06-11 17:15:32,376 | server.py:182 | evaluate_round 10 received 10 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 10 results and 0 failures
INFO flwr 2023-06-11 17:15:32,382 | server.py:147 | FL finished in 30111.154196721
INFO:flwr:FL finished in 30111.154196721
INFO flwr 2023-06-11 17:15:32,386 | app.py:218 | app_fit: losses_distributed [(1, 0.0018555301547050482), (2, 0.0018048945188522344), (3, 0.0017932090640068055), (4, 0.0017723469257354737), (5, 0.0017510627865791326), (6, 0.0017361623764038084), (7, 0.0017284813880920405), (8, 0.001723611843585968), (9, 0.0017159139513969416), (10, 0.0017124316215515138)]
INFO:flwr:app_fit: losses_distributed [(1, 0.0018555301547050482), (2, 0.0018048945188522344), (3, 0.0017932090640068055), (4, 0.0017723469257354737), (5, 0.0017510627865791326), (6, 0.0017361623764038084), (7, 0.0017284813880920405), (8, 0.001723611843585968), (9, 0.0017159139513969416), (10, 0.0017124316215515138)]
INFO flwr 2023-06-11 17:15:32

History (loss, distributed):
	round 1: 0.0018555301547050482
	round 2: 0.0018048945188522344
	round 3: 0.0017932090640068055
	round 4: 0.0017723469257354737
	round 5: 0.0017510627865791326
	round 6: 0.0017361623764038084
	round 7: 0.0017284813880920405
	round 8: 0.001723611843585968
	round 9: 0.0017159139513969416
	round 10: 0.0017124316215515138
History (metrics, distributed, evaluate):
{'accuracy': [(1, 0.3448), (2, 0.3923000000000001), (3, 0.42090000000000005), (4, 0.4463999999999999), (5, 0.4649999999999999), (6, 0.4807000000000001), (7, 0.4972000000000001), (8, 0.513), (9, 0.5232999999999999), (10, 0.5344000000000001)]}